In [ ]:
! pip3 install --upgrade --user google-cloud-aiplatform==1.55.0

In [ ]:
import IPython
from IPython.display import display

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

In [ ]:
import vertexai

PROJECT_ID = ! gcloud config get-value project
PROJECT_ID = PROJECT_ID[0]
LOCATION = "us-central1" # @param {type:"string"}

# define project information manually if the above code didn't work
if PROJECT_ID == "(unset)":
  PROJECT_ID = "[your-project-id]" # @param {type:"string"}

print(PROJECT_ID)

vertexai.init(project=PROJECT_ID, location=LOCATION)



In [ ]:
import base64
import vertexai
from vertexai.generative_models import (GenerativeModel, GenerationConfig,
                                        Image, Part, FinishReason)

def generate(prompt):
  model = GenerativeModel("gemini-1.0-pro-002")
  response = model.generate_content(
      [prompt],
      stream=False,
  )

  return response.text

In [ ]:
prompt = """
Context: You tell Jokes in the Style of Rodney Dangerfield.
Prompt: Tell me a joke about golfing
"""

response = generate(prompt)

print(response)

## Setting the Temperature, Top-K, and Top=P

In [ ]:
def generate(prompt, temperature=0.5, top_k=40, top_p=1):
  model = GenerativeModel("gemini-1.0-pro-002")

  generationConfig = GenerationConfig(
      temperature=temperature,
      top_k=top_k,
      top_p=top_p,
  )

  response = model.generate_content(
      [prompt],
      generation_config=generationConfig,
      stream=False,
  )

  return response.text

In [ ]:
prompt = """
Context: You tell Jokes in the Style of Rodney Dangerfield.
Prompt: Tell me a joke about golfing
"""

response = generate(prompt, temperature=.8, top_p=1, top_k=40)

print(response)

## Basic Chat Code

In [ ]:
model = GenerativeModel("gemini-1.0-pro-002",
    system_instruction=["""Your name is Andrew.
    You tell jokes in the style of Jerry Seinfeld."""])

chat = model.start_chat()

In [ ]:
def multiturn_generate_content(prompt, max_output_tokens= 2048,
                               temperature=1.0, top_p=1):
  generation_config = {
    "max_output_tokens": max_output_tokens,
    "temperature": temperature,
    "top_p": top_p,
  }

  response = chat.send_message(
      [prompt],
      generation_config=generation_config,
      stream=False
  )
  return response.text

In [ ]:
multiturn_generate_content("Hi, who are you?")

In [ ]:
multiturn_generate_content("Tell me a joke about the airport")

In [ ]:
import json

history_data = [obj.__dict__ for obj in chat.history]

print(history_data)
print(chat.history)


In [ ]:
import vertexai.preview.generative_models as generative_models

def safe_multiturn_generate_content(prompt, max_output_tokens= 2048,
                               temperature=1.0, top_p=1):
  generation_config = {
    "max_output_tokens": max_output_tokens,
    "temperature": temperature,
    "top_p": top_p,
  }

  safety_settings = {
    generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
    generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_ONLY_HIGH,
}

  response = chat.send_message(
      [prompt],
      generation_config=generation_config,
      safety_settings=safety_settings,
      stream=False
  )
  return response.text